# 0.0 Import Libs

In [2]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

## Get Catalog list

In [3]:
url = 'https://books.toscrape.com/'
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5),AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
page = requests.get(url, headers=headers)


# BeautifulSoup Objetct
soup = BeautifulSoup(page.text, 'html.parser')

#================== Get Catalog ====================#
catalog_list = soup.find_all('ul',class_='nav nav-list')
catalog = [list(filter(None, p.get_text().split('\n'))) for p in catalog_list]

catalog = [x.strip() for x in catalog[0]]

aux = [i for i in catalog if i.strip()]
catalog_books = pd. DataFrame(aux[1:])
catalog_books.columns = ['catalog']

## All URL Books

In [4]:
#================== Get all url books ====================#
# page size
n_page = soup.find('form', class_='form-horizontal')
n_page = n_page.get_text().split(' ')
n_page = n_page[0].split('\n')[3]
n_page = int(n_page) / 20
page_list = list(np.arange(1, int(n_page)+1,1))

list_url = []

for i in range(len(page_list)):
    url = 'https://books.toscrape.com/catalogue/page-'+ str(page_list[i]) +'.html' 
    page = requests.get(url, headers=headers)
    soup = BeautifulSoup(page.text, 'html.parser')
    product_url = soup.find_all('div',class_='image_container')
    l = [p.find('a').get('href') for p in product_url]
    list_url = list_url + l 

## Get Books Details

In [5]:
#================== Get books details ====================#
#Nome do livro
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5),AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
rate_list = ['One','Two','Three','Four','Five']
cols = ['book_name','book_category','book_price','book_rate','book_stock','book_id']
books = pd.DataFrame(columns=cols)

for i in range(len(list_url)):
    url = 'https://books.toscrape.com/catalogue/' + list_url[i]
    page = requests.get(url, headers=headers)
    soup = BeautifulSoup(page.text, 'html.parser')

    # product name
    product_details = soup.find('div', class_='col-sm-6 product_main')
    name = str(product_details.h1.string)

    # product price
    product_details = soup.find('p', class_='price_color')
    price = product_details.get_text()[1:]

    #product rate
    for p in range(len(rate_list)):
        product_details = soup.find('div', class_='col-sm-6 product_main')

        if product_details.find('p', class_='star-rating '+ str(rate_list[p])):
            aux = product_details.find('p', class_='star-rating '+ str(rate_list[p]))
            rate = aux['class'][1]
        else:
            next

    # product stock
    product_details = soup.find('p', class_='instock availability')
    stock = product_details.get_text().strip()
    
    # product category
    product_details = soup.find('ul', class_='breadcrumb')
    category = list(filter(None, product_details.get_text().split('\n')))[2]
    
    # generate dataframe
    aux = pd.DataFrame([name,category,price,rate,stock]).T
    aux.columns = ['book_name','book_category','book_price','book_rate','book_stock']
    aux['book_id'] = i
    
    books = pd.concat([books, aux], axis=0)

# reset index
books = books.reset_index().drop('index',axis=1)

In [9]:
# Export to csv
books.to_csv('books-list-complete.csv', index=False )